In [1]:
import random


def generate_batches(X_dataset, y_dataset, batch_size):
    ind = 0
    list_index = list(range(0, len(X_dataset)))
    random.shuffle(list_index)
    X_dataset = X_dataset[list_index]
    y_dataset = y_dataset[list_index]
    while ind < X_dataset.shape[0]:
        if ind + batch_size > X_dataset.shape[0]:
            X = X_dataset[ind:]
            y = y_dataset[ind:]
        else:
            X = X_dataset[ind:ind + batch_size]
            y = y_dataset[ind:ind + batch_size]
        ind += batch_size
        yield X, y

In [2]:
class Config:
    IMG_H = 28
    IMG_W = 28
    IMG_C = 1
    IMG_FLATTEN_SHAPE = IMG_H*IMG_W*IMG_C
    BATCH_SIZE = 256
    EPOCHS = 100
    HIDDEN_LAYERS_NEURONS = [16, 16]
    CNN_HIDDEN_LAYERS_NEURONS = [256,128,64]
    LEARNING_RATE = 0.001
    LATENT_DIM = 10
    PLOTTING_SAVING_STEP = 10
    NUMBER_TEST = 10


def recalculate_dim(H, W, C):
    Config.IMG_H = H
    Config.IMG_W = W
    Config.IMG_C = C
    Config.IMG_FLATTEN_SHAPE = H * W * C

In [3]:
import os


def create_class_res_directory(sub_directory):
    if not os.path.exists(sub_directory):
        os.makedirs(sub_directory)
    if not os.path.exists(os.path.join(sub_directory,"plots")):
        os.makedirs(os.path.join(sub_directory,"plots"))


def save_loss_file(_file, itr, dloss, gloss):
    _file.write("%d,%f,%f\n"%(itr,dloss,gloss))


def create_loss_file(sub_directory):
    file = open(os.path.join(sub_directory, 'loss_logs.csv'), 'w')
    file.write('Iteration,Discriminator Loss,Generator Loss\n')
    return file

In [4]:
import numpy as np


def sample_Z(m, n):
    return np.random.uniform(-1, 1, size=[m, n])

In [5]:
import tensorflow.compat.v1 as tf


def _l2normalize(v, eps=1e-12):
    return v / (tf.reduce_sum(v ** 2) ** 0.5 + eps)


def spectral_normed_weight(weights, num_iters=1, update_collection=None, with_sigma=False):
    w_shape = weights.shape.as_list()
    w_mat = tf.reshape(weights, [-1, w_shape[-1]])
    u = tf.get_variable('u', [1, w_shape[-1]],
                        initializer=tf.truncated_normal_initializer(),
                        trainable=False)
    u_ = u
    v_ = tf.ones_like(w_mat)
    for _ in range(num_iters):
        v_ = _l2normalize(tf.matmul(u_, w_mat, transpose_b=True))
        u_ = _l2normalize(tf.matmul(v_, w_mat))

    sigma = tf.squeeze(tf.matmul(tf.matmul(v_, w_mat), u_, transpose_b=True))
    w_mat /= sigma
    if update_collection is None:
        with tf.control_dependencies([u.assign(u_)]):
            w_bar = tf.reshape(w_mat, w_shape)
    else:
        w_bar = tf.reshape(w_mat, w_shape)
        if update_collection != 'NO_OPS':
            tf.add_to_collection(update_collection, u.assign(u_))
    if with_sigma:
        return w_bar, sigma
    else:
        return w_bar


def snconv2d(x, filters, kernel_size=4, strides=2, padding='SAME', use_bias=False, sn=True, name="snconv2d"):
    with tf.variable_scope(name):
        if sn:
            w = tf.get_variable("kernel", shape=[kernel_size, kernel_size, x.get_shape()[-1], filters],
                                initializer=tf.random_normal_initializer(mean=0.0, stddev=0.02),
                                regularizer=None)
            bias = tf.get_variable("bias", [filters], initializer=tf.constant_initializer(0.0))
            x = tf.nn.conv2d(input=x, filter=spectral_normed_weight(w),
                             strides=[1, strides, strides, 1], padding=padding)
            if use_bias:
                x = tf.nn.bias_add(x, bias)

        else:
            x = tf.layers.conv2d(inputs=x, filters=filters,
                                 kernel_size=kernel_size,
                                 strides=strides, use_bias=use_bias, padding=padding)
    return x


def sndeconv2d(x, batch_size, filters, kernel_size=4, strides=2, padding='SAME', use_bias=False, sn=True, name="sndeconv2d"):
    with tf.variable_scope(name):
        x_shape = x.get_shape().as_list()
        output_shape = [batch_size, x_shape[1] * strides, x_shape[2] * strides, filters]
        if sn:
            w = tf.get_variable("kernel", shape=[kernel_size, kernel_size, filters, x.get_shape()[-1]],
                                initializer=tf.random_normal_initializer(mean=0.0, stddev=0.02),
                                regularizer=None)
            x = tf.nn.conv2d_transpose(x, filter=spectral_normed_weight(w), output_shape=output_shape,
                                       strides=[1, strides, strides, 1], padding=padding)

            if use_bias:
                bias = tf.get_variable("bias", [filters], initializer=tf.constant_initializer(0.0))
                x = tf.nn.bias_add(x, bias)

        else:
            x = tf.layers.conv2d_transpose(inputs=x, filters=filters,
                                           kernel_size=kernel_size,
                                           strides=strides, padding=padding, use_bias=use_bias)
        return x

    

In [6]:
import tensorflow_datasets as tfds


def get_mnist_dataset():
    X_train, y_train = tfds.as_numpy(tfds.load('mnist', split='train', batch_size=-1, shuffle_files=True,
                                               as_supervised=True))
    X_train = scale_images(X_train)

    recalculate_dim(X_train.shape[1], X_train.shape[2], X_train.shape[3])

    print(X_train.shape, y_train.shape)
    return X_train, y_train


def scale_images(X_dataset):
    return (X_dataset - 127.5) / 127.5


def flatten_x_dataset(X_dataset):
    return X_dataset.reshape(-1,Config.IMG_FLATTEN_SHAPE)


def reconstruct_x_dataset_img(X_dataset):
    return X_dataset.reshape(-1, Config.IMG_H, Config.IMG_W, Config.IMG_C)

In [7]:
import os


def run_models(X_train, y_train):
    print("********************************************\n")
    print("Mnist Gan Models")
    print("********************************************\n")
    print("Enter \n"
          "1- for model q1 (GanLinearNoNormalization)\n"
          "2- for model q1 (GanLinearWithBatchNormalization)\n"
          "3- for model q1 (Gan2DCnn2DBatchNormalization)\n"
          "4- for model q1 (Gan2DCnnSpectralNormedWeight)\n"
          "else for model q1 (GanLinearNoNormalization)\n")
    model_num = int(input())
    model = None

    if model_num == 2:
        model = GanLinearWithBatchNormalization()
        model.train(X_train, y_train, flatten_images=True)
    elif model_num == 3:
        model = Gan2DCnn2DBatchNormalization()
        model.train(X_train, y_train, flatten_images=False)
    elif model_num == 4:
        model = Gan2DCnnSpectralNormedWeight()
        model.train(X_train, y_train, flatten_images=False)
    else:
        model = GanLinearNoNormalization()
        model.train(X_train, y_train, flatten_images=True)

    Z_batch = sample_Z(Config.NUMBER_TEST, Config.LATENT_DIM)
    G_yhat = model.predict(Z_batch)
    visualize_reconstructed_img(os.path.join(model.sub_directory, "plots"), G_yhat, itr=-1, save_fig=True)

In [8]:
import matplotlib.pyplot as plt
import os


def visualize_reconstructed_img(sub_root, g_plot, itr=-1, save_fig=False):
    num_row = 2
    num_col = 5

    g_plot = reconstruct_x_dataset_img(g_plot)

    fig, ax = plt.subplots(num_row, num_col)
    i = 0

    for row in range(num_row):
        for col in range(num_col):
            plt.sca(ax[row, col])
            plt.imshow(g_plot[i], cmap='gray')
            plt.axis('off')
            i += 1

    plt.title('Samples at Iteration %d' % itr, loc='left')
    if save_fig:
        plt.savefig(os.path.join(sub_root,'iteration_%d.png' % itr))
        plt.close()
    else:
        plt.show()

In [9]:
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(1)


class GanModel(object):
    _sess = None

    def _generator(self, Z, batch_size=tf.Variable(Config.BATCH_SIZE), hsize=Config.HIDDEN_LAYERS_NEURONS, reuse=False):
        pass

    def _discriminator(self, X, batch_size=tf.Variable(Config.BATCH_SIZE), hsize=Config.HIDDEN_LAYERS_NEURONS, reuse=False):
        pass

    def __init__(self):
        tf.reset_default_graph()

        self._G_sample = None
        self._Batch_size = tf.Variable(0)
        self._X = None
        self._Z = None
        self._disc_step = None
        self._disc_loss = None
        self._gen_step = None
        self._gen_loss = None
        self.sub_directory = ""

    def predict(self, Z_batch):
        G_yhat = self._sess.run(self._G_sample, feed_dict={self._Z: Z_batch,
                                                           self._Batch_size: Z_batch.shape[0]})
        return G_yhat

    def train(self, X_train, y_train, flatten_images=True):
        f = create_loss_file(self.sub_directory)
        current_batch_size = Config.BATCH_SIZE
        dloss = 0
        gloss = 0
        for i in range(Config.EPOCHS + 1):
            for X_batch, y_batch in generate_batches(X_train, y_train, Config.BATCH_SIZE):
                if flatten_images:
                    X_batch = flatten_x_dataset(X_batch)
                current_batch_size = X_batch.shape[0]
                Z_batch = sample_Z(current_batch_size, Config.LATENT_DIM)

                _, dloss = self._sess.run([self._disc_step, self._disc_loss], feed_dict={self._X: X_batch,
                                                                                         self._Z: Z_batch,
                                                                                         self._Batch_size: current_batch_size})
                _, gloss = self._sess.run([self._gen_step, self._gen_loss], feed_dict={self._Z: Z_batch,
                                                                                       self._Batch_size: current_batch_size})

            save_fig = False

            if i == int((Config.EPOCHS + 1) / 2) or i == Config.EPOCHS:
                save_fig = True

            if i % Config.PLOTTING_SAVING_STEP == 0:
                print("Iterations: %d\t Discriminator loss: %.4f\t Generator loss: %.4f" % (i, dloss, gloss))
                save_loss_file(f, i, dloss, gloss)

            if save_fig or i % Config.PLOTTING_SAVING_STEP == 0:
                Z_batch = sample_Z(current_batch_size, Config.LATENT_DIM)
                G_yhat = self.predict(Z_batch)
                visualize_reconstructed_img(os.path.join(self.sub_directory, "plots"), G_yhat, itr=i, save_fig=save_fig)

        f.close()

In [10]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


class GanLinearNoNormalization(GanModel):

    def _generator(self, Z, batch_size=tf.Variable(Config.BATCH_SIZE), hsize=Config.HIDDEN_LAYERS_NEURONS, reuse=False):
        with tf.variable_scope("GAN/Generator", reuse=reuse):
            h1 = tf.layers.dense(Z, hsize[0])
            h2 = tf.layers.dense(h1, hsize[1])
            out = tf.layers.dense(h2, Config.IMG_FLATTEN_SHAPE)
        return out

    def _discriminator(self, X, batch_size=tf.Variable(Config.BATCH_SIZE), hsize=Config.HIDDEN_LAYERS_NEURONS, reuse=False):
        with tf.variable_scope("GAN/Discriminator", reuse=reuse):
            h1 = tf.layers.dense(X, hsize[1])
            h2 = tf.layers.dense(h1, hsize[0])
            h3 = tf.layers.dense(h2, Config.IMG_FLATTEN_SHAPE)
            out = tf.layers.dense(h3, 1)
        return out, h3

    def __init__(self):
        super(GanLinearNoNormalization, self).__init__()

        self.sub_directory = "GanLinearNoNormalization"
        create_class_res_directory(self.sub_directory)

        self._X = tf.placeholder(tf.float32, [None, Config.IMG_FLATTEN_SHAPE])
        self._Z = tf.placeholder(tf.float32, [None, Config.LATENT_DIM])

        self._G_sample = self._generator(self._Z)
        r_logits, r_rep = self._discriminator(self._X)
        f_logits, g_rep = self._discriminator(self._G_sample, reuse=True)

        real_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits,
                                                            labels=tf.ones_like(r_logits))
        fake_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,
                                                            labels=tf.zeros_like(f_logits))
        self._disc_loss = tf.reduce_mean(real_loss + fake_loss)
        self._gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,
                                                                                labels=tf.ones_like(f_logits)))

        gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Generator")
        disc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Discriminator")

        self._gen_step = tf.train.RMSPropOptimizer(learning_rate=Config.LEARNING_RATE).minimize(self._gen_loss,
                                                                                                var_list=gen_vars)
        self._disc_step = tf.train.RMSPropOptimizer(learning_rate=Config.LEARNING_RATE).minimize(self._disc_loss,
                                                                                                 var_list=disc_vars)

        self._sess = tf.Session()
        tf.global_variables_initializer().run(session=self._sess)



In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


class GanLinearWithBatchNormalization(GanModel):

    def _generator(self, Z, batch_size=tf.Variable(Config.BATCH_SIZE), hsize=Config.HIDDEN_LAYERS_NEURONS, reuse=False):
        with tf.variable_scope("GAN/Generator", reuse=reuse):
            h1 = tf.layers.dense(Z, hsize[0])
            b1 = tf.layers.batch_normalization(h1)
            h2 = tf.layers.dense(b1, hsize[1])
            out = tf.layers.dense(h2, Config.IMG_FLATTEN_SHAPE)
        return out

    def _discriminator(self, X, batch_size=tf.Variable(Config.BATCH_SIZE), hsize=Config.HIDDEN_LAYERS_NEURONS, reuse=False):
        with tf.variable_scope("GAN/Discriminator", reuse=reuse):
            h1 = tf.layers.dense(X, hsize[1])
            b1 = tf.layers.batch_normalization(h1)
            h2 = tf.layers.dense(b1, hsize[0])
            h3 = tf.layers.dense(h2, Config.IMG_FLATTEN_SHAPE)
            out = tf.layers.dense(h3, 1)
        return out, h3

    def __init__(self):
        super(GanLinearWithBatchNormalization, self).__init__()

        self.sub_directory = "GanLinearWithBatchNormalization"
        create_class_res_directory(self.sub_directory)

        self._X = tf.placeholder(tf.float32, [None, Config.IMG_FLATTEN_SHAPE])
        self._Z = tf.placeholder(tf.float32, [None, Config.LATENT_DIM])

        self._G_sample = self._generator(self._Z)
        r_logits, r_rep = self._discriminator(self._X)
        f_logits, g_rep = self._discriminator(self._G_sample, reuse=True)

        real_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits,
                                                            labels=tf.ones_like(r_logits))
        fake_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,
                                                            labels=tf.zeros_like(f_logits))
        self._disc_loss = tf.reduce_mean(real_loss + fake_loss)
        self._gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,
                                                                                labels=tf.ones_like(f_logits)))

        gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Generator")
        disc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Discriminator")

        self._gen_step = tf.train.RMSPropOptimizer(learning_rate=Config.LEARNING_RATE).minimize(self._gen_loss,
                                                                                                var_list=gen_vars)
        self._disc_step = tf.train.RMSPropOptimizer(learning_rate=Config.LEARNING_RATE).minimize(self._disc_loss,
                                                                                                 var_list=disc_vars)

        self._sess = tf.Session()
        tf.global_variables_initializer().run(session=self._sess)


In [12]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


class Gan2DCnn2DBatchNormalization(GanModel):

    def _generator(self, Z, batch_size=tf.Variable(Config.BATCH_SIZE), hsize=Config.CNN_HIDDEN_LAYERS_NEURONS, reuse=False):
        with tf.variable_scope("GAN/Generator", reuse=reuse):
            x = tf.layers.dense(Z, 7 * 7 * hsize[0], use_bias=False)
            x = tf.layers.batch_normalization(x)
            x = tf.nn.leaky_relu(x)

            x = tf.reshape(x, (tf.shape(Z)[0], 7, 7, hsize[0]))
            # print(x.shape)

            x = tf.layers.conv2d_transpose(x, hsize[1], kernel_size=5, strides=1, padding='same', use_bias=False)
            x = tf.layers.batch_normalization(x)
            x = tf.nn.leaky_relu(x)
            # print(x.shape)

            x = tf.layers.conv2d_transpose(x, hsize[2], kernel_size=5, strides=2, padding='same', use_bias=False)
            x = tf.layers.batch_normalization(x)
            x = tf.nn.leaky_relu(x)
            # print(x.shape)

            x = tf.layers.conv2d_transpose(x, 1, kernel_size=5, strides=2, padding='same', use_bias=False)
            outputs = tf.nn.tanh(x)
            # print(outputs.shape)

        return outputs

    def _discriminator(self, inputs, batch_size=tf.Variable(Config.BATCH_SIZE), hsize=Config.CNN_HIDDEN_LAYERS_NEURONS, reuse=False):
        with tf.variable_scope("GAN/Discriminator", reuse=reuse):
            x = tf.layers.conv2d(inputs, hsize[0], kernel_size=5, strides=2, padding='same')
            x = tf.nn.leaky_relu(x)
            x = tf.layers.dropout(x, 0.3)
            # print(x.shape)

            x = tf.layers.conv2d(x, hsize[1], kernel_size=5, strides=2, padding='same')
            x = tf.nn.leaky_relu(x)
            x = tf.layers.dropout(x, 0.3)
            # print(x.shape)

            x = tf.layers.flatten(x)
            outputs = tf.layers.dense(x, 1)
        return outputs, x

    def __init__(self):
        super(Gan2DCnn2DBatchNormalization, self).__init__()

        self.sub_directory = "Gan2DCnn2DBatchNormalization"
        create_class_res_directory(self.sub_directory)

        self._X = tf.placeholder(tf.float32, [None, Config.IMG_H, Config.IMG_W, Config.IMG_C])
        self._Z = tf.placeholder(tf.float32, [None, Config.LATENT_DIM])

        self._G_sample = self._generator(self._Z)
        r_logits, r_rep = self._discriminator(self._X)
        f_logits, g_rep = self._discriminator(self._G_sample, reuse=True)

        real_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits,
                                                            labels=tf.ones_like(r_logits))
        fake_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,
                                                            labels=tf.zeros_like(f_logits))
        self._disc_loss = tf.reduce_mean(real_loss + fake_loss)
        self._gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,
                                                                                labels=tf.ones_like(f_logits)))

        gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Generator")
        disc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Discriminator")

        self._gen_step = tf.train.RMSPropOptimizer(learning_rate=Config.LEARNING_RATE).minimize(self._gen_loss,
                                                                                                var_list=gen_vars)
        self._disc_step = tf.train.RMSPropOptimizer(learning_rate=Config.LEARNING_RATE).minimize(self._disc_loss,
                                                                                                 var_list=disc_vars)

        self._sess = tf.Session()
        tf.global_variables_initializer().run(session=self._sess)

In [13]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


class Gan2DCnnSpectralNormedWeight(GanModel):

    def _generator(self, Z, batch_size=tf.Variable(Config.BATCH_SIZE), hsize=Config.CNN_HIDDEN_LAYERS_NEURONS, reuse=False):
        with tf.variable_scope("GAN/Generator", reuse=reuse):
            x = tf.layers.dense(Z, 7 * 7 * hsize[0], use_bias=False)
            # x = tf.layers.batch_normalization(x)
            x = tf.nn.leaky_relu(x)

            x = tf.reshape(x, (tf.shape(Z)[0], 7, 7, hsize[0]))
            # print(x.shape)

            x = sndeconv2d(x, self._Batch_size, hsize[1], kernel_size=5, strides=1, padding='SAME', use_bias=False, sn=True,
                           name="sndeconv2d_1")
            # x = tf.layers.batch_normalization(x)
            x = tf.nn.leaky_relu(x)
            # print(x.shape)

            x = sndeconv2d(x, self._Batch_size, hsize[2], kernel_size=5, strides=2, padding='SAME', use_bias=False, sn=True,
                           name="sndeconv2d_2")
            # x = tf.layers.batch_normalization(x)
            x = tf.nn.leaky_relu(x)
            # print(x.shape)

            x = sndeconv2d(x, self._Batch_size, 1, kernel_size=5, strides=2, padding='SAME', use_bias=False, sn=True,
                           name="sndeconv2d_3")
            outputs = tf.nn.tanh(x)
        return outputs

    def _discriminator(self, inputs, batch_size=tf.Variable(Config.BATCH_SIZE), hsize=Config.CNN_HIDDEN_LAYERS_NEURONS, reuse=False):
        with tf.variable_scope("GAN/Discriminator", reuse=reuse):
            x = snconv2d(inputs, hsize[0], kernel_size=5, strides=2, padding='SAME', use_bias=False, sn=True,
                         name="snconv2d_1")
            x = tf.nn.leaky_relu(x)
            x = tf.layers.dropout(x, 0.3)
            # print(x.shape)

            x = snconv2d(x, hsize[1], kernel_size=5, strides=2, padding='SAME', use_bias=False, sn=True,
                         name="snconv2d_2")
            x = tf.nn.leaky_relu(x)
            x = tf.layers.dropout(x, 0.3)

            x = tf.layers.flatten(x)
            outputs = tf.layers.dense(x, 1)
        return outputs, x

    def __init__(self):
        super(Gan2DCnnSpectralNormedWeight, self).__init__()

        self.sub_directory = "Gan2DCnnSpectralNormedWeight"
        create_class_res_directory(self.sub_directory)

        self._X = tf.placeholder(tf.float32, [None, Config.IMG_H, Config.IMG_W, Config.IMG_C])
        self._Z = tf.placeholder(tf.float32, [None, Config.LATENT_DIM])
        self._Batch_size = tf.Variable(Config.BATCH_SIZE)

        self._G_sample = self._generator(self._Z, self._Batch_size)
        r_logits, r_rep = self._discriminator(self._X, self._Batch_size)
        f_logits, g_rep = self._discriminator(self._G_sample, self._Batch_size, reuse=True)

        real_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits,
                                                            labels=tf.ones_like(r_logits))
        fake_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,
                                                            labels=tf.zeros_like(f_logits))
        self._disc_loss = tf.reduce_mean(real_loss + fake_loss)
        self._gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,
                                                                                labels=tf.ones_like(f_logits)))

        gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Generator")
        disc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Discriminator")

        self._gen_step = tf.train.RMSPropOptimizer(learning_rate=Config.LEARNING_RATE).minimize(self._gen_loss,
                                                                                                var_list=gen_vars)
        self._disc_step = tf.train.RMSPropOptimizer(learning_rate=Config.LEARNING_RATE).minimize(self._disc_loss,
                                                                                                 var_list=disc_vars)

        self._sess = tf.Session()
        tf.global_variables_initializer().run(session=self._sess)

In [14]:
if __name__ == "__main__":
    X_train, y_train = get_mnist_dataset()
    run_models(X_train, y_train)
